# Hyper-Param and Iterative jobs

MLRun support iterative tasks for automatic and distributed execution of many tasks with variable parameters, this can be used for various tasks such as:
* Parallel loading and preparation of many data objects
* Model training with different parameter sets and/or algorithms
* Parallel testing with many test vector options 

MLRun iterations can be viewed as child runs under the main task/run, each child run will get a set of parameters which will be computed/selected from the input hyper parameters based on the chosen strategy (Grid, List, Random or Custom).

The hyper parameters and options are specified in the `task` or the `function.run()` command through the `hyperparams` (for hyper param values) and `hyper_param_options` (for {py:class}`~mlrun.model.HyperParamOptions`) properties, see examples below. hyper parameters can also be loaded directly from a CSV or Json file (by setting the `param_file` hyper option).

The hyper params are specified as a struct of `key: list` values for example: `{"p1": [1,2,3], "p2": [10,20]}`, the values can be of any type (int, string, float, ..), the list are used to compute the parameter combinations using one of the following strategies: 
1. Grid Search (`grid`) - running all the parameter combinations
2. Random (`random`) - running a sampled set from all the parameter combinations
3. List (`list`) - running the first parameter from each list followed by the 2nd from each list and so on, note that all the lists must be of equal size.

MLRun also support a 4th `custom` option which allow determining the parameter combination per run programmatically 

You can specify a selection criteria to select the best run among the different child runs by setting the `selector` option, this will mark that result as the parent (iteration 0) result, and mark the best result in the user interface.

You can also specify the `stop_condition` to stop execution of child runs when some criteria based on the returned results is met (for example `stop_condition="accuracy>=0.9"`)

## Examples

**Base dummy function:**

In [ ]:
import mlrun

In [2]:
def hyper_func(context, p1, p2):
    print(f"p1={p1}, p2={p2}, result={p1 * p2}")
    context.log_result("mul", p1 * p2)

### Grid Search (default)

In [25]:
grid_params = {"p1": [2,4,1], "p2": [10,20]}
task = mlrun.new_task("grid-demo").with_hyper_params(grid_params, selector="max.mul")
run = mlrun.new_function().run(task, handler=hyper_func)

> 2021-03-07 22:36:44,926 [info] starting run grid-demo uid=649337c098f64ea9a91966bed2539b95 DB=http://mlrun-api:8080
p1=2, p2=10, result=20
p1=4, p2=10, result=40
p1=1, p2=10, result=10
p1=2, p2=20, result=40
p1=4, p2=20, result=80
p1=1, p2=20, result=20
> 2021-03-07 22:36:45,759 [info] best iteration=5, used criteria max.mul


project,uid,iter,start,state,name,labels,inputs,parameters,results,artifacts
default,...d2539b95,0,Mar 07 22:36:44,completed,grid-demo,v3io_user=adminkind=handlerowner=admin,,,best_iteration=5mul=80,iteration_results


to track results use .show() or .logs() or in CLI: 
!mlrun get run 649337c098f64ea9a91966bed2539b95 --project default , !mlrun logs 649337c098f64ea9a91966bed2539b95 --project default
> 2021-03-07 22:36:45,927 [info] run executed, status=completed


**UI Screenshot:**
<br><br>
<img src="_static/images/hyper-params.png" alt="hyper-params" width="800"/>


### Random Search
in random search MLRun will choose random parameter combinations, we limit the number of combinations using the `max_iterations` attribute

In [26]:
grid_params = {"p1": [2,4,1,3], "p2": [10,20,30]}
task = mlrun.new_task("random-demo").with_hyper_params(grid_params, selector="max.mul", strategy="random", max_iterations=4)
run = mlrun.new_function().run(task, handler=hyper_func)

> 2021-03-07 22:36:51,065 [info] starting run random-demo uid=f9e87f6d80e948d0ab5971d21661234a DB=http://mlrun-api:8080
p1=2, p2=10, result=20
p1=4, p2=30, result=120
p1=1, p2=20, result=20
p1=3, p2=10, result=30
> 2021-03-07 22:36:51,649 [info] best iteration=2, used criteria max.mul


project,uid,iter,start,state,name,labels,inputs,parameters,results,artifacts
default,...1661234a,0,Mar 07 22:36:51,completed,random-demo,v3io_user=adminkind=handlerowner=admin,,,best_iteration=2mul=120,iteration_results


to track results use .show() or .logs() or in CLI: 
!mlrun get run f9e87f6d80e948d0ab5971d21661234a --project default , !mlrun logs f9e87f6d80e948d0ab5971d21661234a --project default
> 2021-03-07 22:36:51,822 [info] run executed, status=completed


### List Search

In this example we also show how to use the `stop_condition` option

In [29]:
list_params = {"p1": [2,3,7,4,5], "p2": [15,10,10,20,30]}
task = mlrun.new_task("list-demo").with_hyper_params(list_params, selector="max.mul", strategy="list", stop_condition="mul>=70")
run = mlrun.new_function().run(task, handler=hyper_func)

> 2021-03-07 22:39:11,921 [info] starting run list-demo uid=f4df1aae4f4942a4bfff18d55138a4d3 DB=http://mlrun-api:8080
p1=2, p2=15, result=30
p1=3, p2=10, result=30
p1=7, p2=10, result=70
> 2021-03-07 22:39:12,382 [info] reached early stop condition (mul>=70), stopping iterations!
> 2021-03-07 22:39:12,385 [info] best iteration=3, used criteria max.mul


project,uid,iter,start,state,name,labels,inputs,parameters,results,artifacts
default,...5138a4d3,0,Mar 07 22:39:11,completed,list-demo,v3io_user=adminkind=handlerowner=admin,,,best_iteration=3mul=70,iteration_results


to track results use .show() or .logs() or in CLI: 
!mlrun get run f4df1aae4f4942a4bfff18d55138a4d3 --project default , !mlrun logs f4df1aae4f4942a4bfff18d55138a4d3 --project default
> 2021-03-07 22:39:12,581 [info] run executed, status=completed


### Custom Iterator

We can define a child iteration context under the parent/main run, the child run will be logged independently  

In [10]:
def handler(context: mlrun.MLClientCtx, param_list):
    best_mul = total = 0
    for param in param_list:
        with context.get_child_context(**param) as child:
            hyper_func(child, **child.parameters)
            mul = child.results['mul']
            total += mul
            if mul > best_mul:
                child.mark_as_best()
                best_mul = mul

    # log result at the parent
    context.log_result('avg_mul', total / len(param_list))

In [11]:
param_list = [{"p1":2, "p2":10}, {"p1":3, "p2":30}, {"p1":4, "p2":7}]
run = mlrun.new_function().run(handler=handler, params={"param_list": param_list})

> 2021-03-07 23:08:10,391 [info] starting run mlrun-ff1b06-handler uid=70c45c331ba5478897537df1c7eb3abf DB=http://mlrun-api:8080
p1=2, p2=10, result=20
p1=3, p2=30, result=90
p1=4, p2=7, result=28


project,uid,iter,start,state,name,labels,inputs,parameters,results,artifacts
default,...c7eb3abf,0,Mar 07 23:08:10,completed,mlrun-ff1b06-handler,v3io_user=adminkind=handlerowner=adminhost=jupyter-65c78cc479-jtz4j,,"param_list=[{'p1': 2, 'p2': 10}, {'p1': 3, 'p2': 30}, {'p1': 4, 'p2': 7}]",best_iteration=2mul=90avg_mul=46.0,


to track results use .show() or .logs() or in CLI: 
!mlrun get run 70c45c331ba5478897537df1c7eb3abf --project default , !mlrun logs 70c45c331ba5478897537df1c7eb3abf --project default
> 2021-03-07 23:08:11,422 [info] run executed, status=completed


In [ ]:
grid_params = {"p2": [2,4,1], "p3": [10,20]}
task = mlrun.new_task(params={"p1": 8})
task.with_hyper_params(
    grid_params, selector="max.mul", strategy="grid", parallel_runs=4, dask_cluster_uri=uri, teardown_dask=True
)

## Parallel Execution Over Containers

When working with compute intensive or long running tasks we would like to run our iterations over a cluster of containers, on the same time we dont want to bring up too many containers and rather limit the number of parallel tasks.

MLRun support distribution of the child runs over a Dask cluster, this is handled automatically by MLRun, the user only need to specify the Dask configuration and the level of parallelism. The execution can be controlled from the client/notebook, or can have a job (immediate or scheduled) which control the execution.

In the following example we create a new function and execute the parent/controller as an MLRun `job` and the different child runs over a Dask cluster (MLRun Dask function).

In [12]:
# mark the start of a code section which will be be sent to the job
# nuclio: start-code

In [20]:
import socket
import pandas as pd
def hyper_func2(context, data, p1, p2, p3):
    print(data.as_df().head())
    print(f"p2={p2}, p3={p3}, r1={p2 * p3} at {socket.gethostname()}")
    context.log_result("r1", p2 * p3)
    raw_data = {
        "first_name": ["Jason", "Molly", "Tina", "Jake", "Amy"],
        "age": [42, 52, 36, 24, 73],
        "testScore": [25, 94, 57, 62, 70],
    }
    df = pd.DataFrame(raw_data, columns=["first_name", "age", "testScore"])
    context.log_dataset("mydf", df=df, stats=True)

In [2]:
# nuclio: end-code

### Define a Dask Cluster (using MLRun serverless Dask)

In [25]:
dask_cluster = mlrun.new_function("dask-cluster", kind='dask', image='mlrun/ml-models')
dask_cluster.apply(mlrun.mount_v3io())        # add volume mounts
dask_cluster.spec.service_type = "NodePort"   # open interface to the dask UI dashboard
dask_cluster.spec.replicas = 2                # define two containers
uri = dask_cluster.save()

### Define the Parallel Work

We set the `parallel_runs` attribute to indicate how many child tasks to run in parallel, and set the `dask_cluster_uri` to point to our dask cluster (if we dont set the cluster uri it will use dask local), we can also set the `teardown_dask` flag to indicate we want to free up all the dask resources after completion.

In [26]:
uri

'db://default/dask-cluster'

In [27]:
grid_params = {"p2": [2,1,4,1], "p3": [10,20]}
task = mlrun.new_task(params={"p1": 8}, inputs={'data': 'https://s3.wasabisys.com/iguazio/data/iris/iris_dataset.csv'})
task.with_hyper_params(
    grid_params, selector="r1", strategy="grid", parallel_runs=4, dask_cluster_uri=uri, teardown_dask=True
)

**Define a job that will take our code (using code_to_function) and run it over the cluster**

In [28]:
fn = mlrun.code_to_function(name='hyper-tst', kind='job', image='mlrun/ml-models')

In [30]:
run = fn.run(task, handler=hyper_func2)

> 2021-03-07 23:44:18,545 [info] starting run hyper-tst-hyper_func2 uid=a23e0171c6ae441a8f3463ac33cf09ae DB=http://mlrun-api:8080
> 2021-03-07 23:44:25,588 [info] trying dask client at: tcp://mlrun-dask-cluster-e4a98649-4.default-tenant:8786
> 2021-03-07 23:44:25,630 [info] using remote dask scheduler (mlrun-dask-cluster-e4a98649-4) at: tcp://mlrun-dask-cluster-e4a98649-4.default-tenant:8786


/conda/lib/python3.7/site-packages/distributed/client.py:1129: VersionMismatchWarning: Mismatched versions found

+-------------+--------+-----------+---------+
| Package     | client | scheduler | workers |
+-------------+--------+-----------+---------+
| blosc       | 1.7.0  | 1.10.2    | None    |
| distributed | 2.30.0 | 2.30.1    | None    |
| lz4         | 3.1.0  | 3.1.3     | None    |
| msgpack     | 1.0.0  | 1.0.2     | None    |
| toolz       | 0.11.1 | 0.10.0    | None    |
| tornado     | 6.0.4  | 6.0.3     | None    |
+-------------+--------+-----------+---------+
Notes: 
-  msgpack: Variation is ok, as long as everything is above 0.6
  warnings.warn(version_module.VersionMismatchWarning(msg[0]["warning"]))


> --------------- Iteration: (1) ---------------
   sepal length (cm)  sepal width (cm)  ...  petal width (cm)  label
0                5.1               3.5  ...               0.2      0
1                4.9               3.0  ...               0.2      0
2                4.7               3.2  ...               0.2      0
3                4.6               3.1  ...               0.2      0
4                5.0               3.6  ...               0.2      0

[5 rows x 5 columns]
p2=2, p3=10, r1=20 at mlrun-dask-cluster-e4a98649-4clszk

> --------------- Iteration: (2) ---------------
   sepal length (cm)  sepal width (cm)  ...  petal width (cm)  label
0                5.1               3.5  ...               0.2      0
1                4.9               3.0  ...               0.2      0
2                4.7               3.2  ...               0.2      0
3                4.6               3.1  ...               0.2      0
4                5.0               3.6  ...               0.2  

project,uid,iter,start,state,name,labels,inputs,parameters,results,artifacts
default,...33cf09ae,0,Mar 07 23:44:18,completed,hyper-tst-hyper_func2,v3io_user=adminkind=owner=admin,data,p1=8,best_iteration=7r1=80,mydfiteration_results


to track results use .show() or .logs() or in CLI: 
!mlrun get run a23e0171c6ae441a8f3463ac33cf09ae --project default , !mlrun logs a23e0171c6ae441a8f3463ac33cf09ae --project default
> 2021-03-07 23:44:34,667 [info] run executed, status=completed
